In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append('../')  # Add the parent folder to the Python path

from sql_functions import get_dataframe  # Import the function

main_cities = [
    "New York, NY",
    "Boston, MA",
    "Washington, DC",
    "Newark, NJ",
]

side_cities = [
    "Miami, FL",
    "Houston, TX",
    "San Francisco, CA",
    "Seattle, WA"
]


In [2]:
flights_2015 = get_dataframe('SELECT * FROM group3.flights_2015')
flights_2016 = get_dataframe('SELECT * FROM group3.flights_2016')
flights_2017 = get_dataframe('SELECT * FROM group3.flights_2017')

In [5]:
flights_2016

,year,month,day,flight_date,dep_delay,arr_delay,origin,destination,cancelled,cancellation_code,diverted,cancellation_code_info,had_delay
0,2016,1,1,2016-01-01,-4.0,8.0,"Seattle, WA","New York, NY",0.0,None,0.0,not cancelled,1
1,2016,1,2,2016-01-02,-8.0,11.0,"Seattle, WA","New York, NY",0.0,None,0.0,not cancelled,1
2,2016,1,3,2016-01-03,-2.0,-15.0,"Seattle, WA","New York, NY",0.0,None,0.0,not cancelled,1
3,2016,1,4,2016-01-04,-8.0,-11.0,"Seattle, WA","New York, NY",0.0,None,0.0,not cancelled,1
4,2016,1,5,2016-01-05,-2.0,-3.0,"Seattle, WA","New York, NY",0.0,None,0.0,not cancelled,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1949492,2016,12,30,2016-12-30,3.0,-18.0,"New York, NY","Fort Myers, FL",0.0,None,0.0,not cancelled,1
1949493,2016,12,30,2016-12-30,1.0,-21.0,"Washington, DC","Atlanta, GA",0.0,None,0.0,not cancelled,1
1949494,2016,12,30,2016-12-30,-4.0,-21.0,"New York, NY","Fort Lauderdale, FL",0.0,None,0.0,not cancelled,1
1949495,2016,12,30,2016-12-30,-7.0,-33.0,"New York, NY","New Orleans, LA",0.0,None,0.0,not cancelled,1


In [4]:
flights_2016[flights_2016[main]]

NameError: name 'main' is not defined